In [1]:
import sys
sys.path.insert(0,'..')

In [2]:
#from food.psql import *
from mytools.tools import get_logger
# logger = get_logger(engine,'bot_logs','food')
# logger.debug({'msg':'starting bot'})
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.utils.callback_data import CallbackData
import typing
import numpy as np
import io


from food.paths import *


API_TOKEN = bot_token

from food.paths import *
#from food.search import *
import pandas as  pd
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

import nest_asyncio
nest_asyncio.apply()


In [3]:
from food.psql import *

from food.search import *

dev
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.


/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (UnsupervisedMetrics). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


/home/dima/FoodSeg103-Benchmark-v1/checkpoints/SETR_Naive_ReLeM


/home/dima/FoodSeg103-Benchmark-v1/mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


load pre-trained weight from imagenet21k


/home/dima/mytools/mytools/visual.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])


In [4]:
def plot_nutrition(masks):
    attributes = ['energy','protein','carb','fat']
    nutrition ={}
    for m,a in zip(masks,attributes): nutrition[a] = int(m[m!=0].mean())
    return pd.DataFrame(nutrition,index = ['']).to_string()

In [5]:
def image2file_obj(img):
    o = io.BytesIO()
    f = img.format if img.format else 'jpeg'
    img.save(o, format=f)
    return o.getvalue()

In [6]:
async def async_image2file_obj(img):
    return image2file_obj(img)

async def async_search(url):
    return search(url)

async def async_visualize_array(*args,**kwargs):
    return visualize_array(*args,**kwargs)

In [7]:
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

In [8]:
dishes_table = Dishes.__table__

add_dish_cb     = CallbackData('add dish', 'action')
edit_dish_cb    = CallbackData('edit_dish', 'action')

In [13]:
def get_keyboard(t, unit = None):
    markup = types.InlineKeyboardMarkup()
    if t == 'add dish' :  
        markup.add(types.InlineKeyboardButton('add dish', callback_data=add_dish_cb.new(action='add_dish')))
         

    elif t == 'edit_dish':

        btns_text = ('remove',)
        markup.add(*(types.InlineKeyboardButton(text, callback_data=edit_dish_cb.new(action=text)) for text in btns_text))

    return markup 

Cause exception while getting updates.
Traceback (most recent call last):
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/bot/api.py", line 139, in make_request
    async with session.post(url, data=req, **kwargs) as response:
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiohttp/client.py", line 1138, in __aenter__
    self._resp = await self._coro
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiohttp/client.py", line 559, in _request
    await resp.start(conn)
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 898, in start
    message, payload = await protocol.read()  # type: ignore[union-attr]
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiohttp/streams.py", line 616, in read
    await self._waiter
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/asyncio/futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File

In [10]:
@dp.message_handler(content_types=ContentType.PHOTO,state='*')
async def process_photo(message: types.Message, state: FSMContext):

    
    await types.ChatActions.upload_photo()
    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    # image_url    = await photo.get_url()
    image_url      = f'https://dima.grankin.eu/reference_images/{photo["file_id"]}'
    
    img,clip_df,masks,urls,stats = await async_search(image_url)
    

    img_o = await async_image2file_obj(img)
    await message.reply_photo(img_o,caption=f'{plot_nutrition(masks)} \n', reply_markup=get_keyboard('add dish'))

In [11]:
#add_dish pushed
@dp.callback_query_handler(add_dish_cb.filter(action=['add_dish']))
async def add_dish(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    
    # calc food score

    msg = query.to_python()['message']['caption']
    msg = msg.split('\xa0')[0] if '\xa0' in msg else msg
    # msg = f"{msg}\n \xa0 some other appendix"


    await bot.edit_message_caption(query.from_user.id,
                                    query.message.message_id,
                                    caption = msg,
                                    reply_markup=get_keyboard('edit_dish'))

In [ ]:
if __name__ == '__main__': executor.start_polling(dp)

Cause exception while getting updates.
Traceback (most recent call last):
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/dispatcher/dispatcher.py", line 381, in start_polling
    updates = await self.bot.get_updates(
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/bot/bot.py", line 110, in get_updates
    result = await self.request(api.Methods.GET_UPDATES, payload)
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/bot/base.py", line 231, in request
    return await api.make_request(await self.get_session(), self.server, self.__token, method, data, files,
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/bot/api.py", line 140, in make_request
    return check_result(method, response.content_type, response.status, await response.text())
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/bot/api.py", line 119, in check_result
    exceptions.ConflictError.detect(descripti

In [20]:
query = q

In [23]:
bot.edit_message_caption?

Signature:
bot.edit_message_caption(
    chat_id: 'typing.Union[base.Integer, base.String, None]' = None,
    message_id: 'typing.Optional[base.Integer]' = None,
    inline_message_id: 'typing.Optional[base.String]' = None,
    caption: 'typing.Optional[base.String]' = None,
    parse_mode: 'typing.Optional[base.String]' = None,
    caption_entities: 'typing.Optional[typing.List[types.MessageEntity]]' = None,
    reply_markup: 'typing.Union[types.InlineKeyboardMarkup, None]' = None,
) -> 'types.Message or base.Boolean'
Docstring:
Use this method to edit captions of messages sent by the bot or via the bot (for inline bots).

Source: https://core.telegram.org/bots/api#editmessagecaption

:param chat_id: Required if inline_message_id is not specified
    Unique identifier for the target chat or username of the target channel
:type chat_id: :obj:`typing.Union[base.Integer, base.String, None]`

:param message_id: Required if inline_message_id is not specified. Identifier of the sent message

<Message {"message_id": 2628, "from": {"id": 5091011572, "is_bot": true, "first_name": "calorie_bot", "username": "food_classifier_bot"}, "chat": {"id": 78882798, "first_name": "Dmitry", "last_name": "Grankin", "username": "dmitry_grankin", "type": "private"}, "date": 1659544989, "edit_date": 1659545079, "photo": [{"file_id": "AgACAgIAAxkDAAIKRGLqpff3cTl_AAEIBk4CXYqRpV3Q_gACEb0xG9sOWEuYoKV9mZeLvgEAAwIAA3MAAykE", "file_unique_id": "AQADEb0xG9sOWEt4", "file_size": 1083, "width": 90, "height": 51}, {"file_id": "AgACAgIAAxkDAAIKRGLqpff3cTl_AAEIBk4CXYqRpV3Q_gACEb0xG9sOWEuYoKV9mZeLvgEAAwIAA20AAykE", "file_unique_id": "AQADEb0xG9sOWEty", "file_size": 13000, "width": 320, "height": 180}, {"file_id": "AgACAgIAAxkDAAIKRGLqpff3cTl_AAEIBk4CXYqRpV3Q_gACEb0xG9sOWEuYoKV9mZeLvgEAAwIAA3gAAykE", "file_unique_id": "AQADEb0xG9sOWEt9", "file_size": 54516, "width": 800, "height": 450}, {"file_id": "AgACAgIAAxkDAAIKRGLqpff3cTl_AAEIBk4CXYqRpV3Q_gACEb0xG9sOWEuYoKV9mZeLvgEAAwIAA3kAAykE", "file_unique_id": "AQAD

In [15]:
query.to_python()['message']['caption']

'energy  protein  carb  fat\n     169        9    16    7'

In [17]:
msg = query.to_python()['message']['caption']
msg = msg.split('\xa0')[0] if '\xa0' in msg else msg
msg = f"{msg}\n \xa0 please choose unit for your food weight measurement"

print ('here')


await bot.edit_message_caption(msg,
                            query.from_user.id,
                            query.message.message_id,
                            reply_markup=get_keyboard('edit_dish'))

here


ChatNotFound: Chat not found

In [ ]:
bot.edit_message_caption

In [15]:
msg = query.to_python()['message']['text']
msg = msg.split('\xa0')[0] if '\xa0' in msg else msg
msg = f"{msg}\n \xa0 please choose unit for your food weight measurement"

Task exception was never retrieved
future: <Task finished name='Task-15' coro=<Dispatcher._process_polling_updates() done, defined at /home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/dispatcher/dispatcher.py:407> exception=KeyError('text')>
Traceback (most recent call last):
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/asyncio/tasks.py", line 258, in __step
    result = coro.throw(exc)
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/dispatcher/dispatcher.py", line 415, in _process_polling_updates
    for responses in itertools.chain.from_iterable(await self.process_updates(updates, fast)):
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/aiogram/dispatcher/dispatcher.py", line 235, in process_updates
    return await asyncio.gather(*tasks)
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/asyncio/tasks.py", line 328, in __wakeup
    future.result()
  File "/home/dima/anaconda3/envs/f4/lib/python3.9/asyncio/tasks.py", line 

KeyError: 'text'